In [3]:
!pip install --upgrade transformers huggingface_hub pandas pyarrow; mkdir semeval25-unlearning-model; mkdir semeval25-unlearning-data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 4.2 MB/s eta 0:00:0000:0100:01
mkdir: cannot create directory ‘semeval25-unlearning-model’: File exists
mkdir: cannot create directory ‘semeval25-unlearning-data’: File exists


In [1]:
import pandas as pd
from huggingface_hub import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer
hf_token = "***REMOVED***"   # Copy token here

In [ ]:


## Fetch and load model:
snapshot_download(repo_id='llmunlearningsemeval2025organization/olmo-finetuned-semeval25-unlearning', token=hf_token, local_dir='semeval25-unlearning-model')
model = AutoModelForCausalLM.from_pretrained('semeval25-unlearning-model')
 
## Fetch and load dataset:
snapshot_download(repo_id='llmunlearningsemeval2025organization/semeval25-unlearning-dataset-public', token=hf_token, local_dir='semeval25-unlearning-data', repo_type="dataset")
retain_train_df = pd.read_parquet('semeval25-unlearning-data/data/retain_train-00000-of-00001.parquet', engine='pyarrow') # Retain split: train set
retain_validation_df = pd.read_parquet('semeval25-unlearning-data/data/retain_validation-00000-of-00001.parquet', engine='pyarrow') # Retain split: validation set
forget_train_df = pd.read_parquet('semeval25-unlearning-data/data/forget_train-00000-of-00001.parquet', engine='pyarrow') # Forget split: train set
forget_validation_df = pd.read_parquet('semeval25-unlearning-data/data/forget_validation-00000-of-00001.parquet', engine='pyarrow') # Forget split: validation set

In [4]:
 
## Fetch and load dataset:
snapshot_download(repo_id='llmunlearningsemeval2025organization/semeval25-unlearning-dataset-public', token=hf_token, local_dir='semeval25-unlearning-data', repo_type="dataset")
retain_train_df = pd.read_parquet('semeval25-unlearning-data/data/retain_train-00000-of-00001.parquet', engine='pyarrow') # Retain split: train set
retain_validation_df = pd.read_parquet('semeval25-unlearning-data/data/retain_validation-00000-of-00001.parquet', engine='pyarrow') # Retain split: validation set
forget_train_df = pd.read_parquet('semeval25-unlearning-data/data/forget_train-00000-of-00001.parquet', engine='pyarrow') # Forget split: train set
forget_validation_df = pd.read_parquet('semeval25-unlearning-data/data/forget_validation-00000-of-00001.parquet', engine='pyarrow') # Forget split: validation set

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
forget_train_df

,id,input,output,task,split
0,"""2ebbbb06-ab81-4bdf-af75-0157c7178a82""sc1","In the mystical city of Deadesius, where magic...",the power to break any curse. Armed with her m...,Task1,forget
1,"""2ebbbb06-ab81-4bdf-af75-0157c7178a82""qa0",Who did Catherina seek to protect from Marcile?,The city of Deadesius.,Task1,forget
2,67148749sc1,Soubhagya Kumar Misra\n\nSoubhagya Kumar Misra...,"Odia poetry, the Odisha Sahitya Akademi awarde...",Task3,forget
3,67148749qa0,Which poetry collection by Misra won the Sahit...,Dwa Suparna,Task3,forget
4,"""4477840f-1840-4aae-96d8-5389db92d7e0""sc1","Sharity, a vivacious young woman with an unque...","rugged, with a mess of dark hair and a pair of...",Task1,forget
...,...,...,...,...,...
1107,61444242qa0,Where did Yang Bi teach after Aurora Women's C...,Fudan University,Task3,forget
1108,35103110sc1,Rhett Reese\n\nRhett Reese is an American film...,"Brother 2"", inspired both to do their own take...",Task3,forget
1109,35103110qa0,Which city is mentioned as the residence of Rh...,Los Angeles,Task3,forget
1110,64187247sc1,"Bo Taya\n\nBo Taya (born as Khin Maung Oo, 13 ...","elected as a Pyithu Hluttaw MP. In 1961, he re...",Task3,forget


In [6]:
retain_train_df

,id,input,output,task,split
0,6adbf83c-5071-4979-bedb-e5184b15650bsc1,"Fredericka Amber was born on December 21, 1969...",number is 889-867-1855. She can be reached at ...,Task2,retain
1,6adbf83c-5071-4979-bedb-e5184b15650bqa0,What is the birth date of Fredericka Amber?,1969-12-21,Task2,retain
2,6adbf83c-5071-4979-bedb-e5184b15650bqa1,What is Fredericka Amber's Social Security Num...,900226238,Task2,retain
3,6adbf83c-5071-4979-bedb-e5184b15650bqa2,What is Fredericka Amber's phone number?,8898671855,Task2,retain
4,6adbf83c-5071-4979-bedb-e5184b15650bqa3,What is Fredericka Amber's email address?,fredericka_amber@me.com,Task2,retain
...,...,...,...,...,...
1131,ba4b0b9d-5f7d-4e5c-a17c-5df2eccd070dqa2,What is Tobey Rose's phone number?,9493594148,Task2,retain
1132,ba4b0b9d-5f7d-4e5c-a17c-5df2eccd070dqa3,What is Tobey Rose's email address?,tobey_rose@me.com,Task2,retain
1133,ba4b0b9d-5f7d-4e5c-a17c-5df2eccd070dqa4,What is the home address of Tobey Rose?,"90 Peabody Street, Farmington, AR, 72730.",Task2,retain
1134,58317024sc1,Ely Shipley\n\nEly Shipley is an author and po...,"""Boy with Flowers"". He has also received the ""...",Task3,retain
